<a href="https://colab.research.google.com/github/Reustlin/technical_task/blob/main/task_categorii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создайте модель, обрабатывающую фрагмент текста и определяющую
какой вид продукции в нём содержится.

Виды продукции (брать только виды продукции, для которых в датасете есть не менее 500 примеров):

* Баранина
* Ягнятина
* Индейка
* Говядина
* Свинина
* Кура
* Цыпленок
* Гусь
* Буйволятина
* Оленина
* Конина
* Телятина
* Кролик
* Утка
* Куропатка
* Перепел
* Глухарь
* Страус
* Заяц
* Кенгуру
* Изюбр
* Кабан
* Коза
* Косуля
* Лось
* Марал
* Медвежатина
* Бобер
* Цесарка
* Нутрия
* Рябчик
* Тетерев
* Фазан
* Як




In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import pymorphy2
import nltk

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/meatinfo.csv', sep = ';')

In [ ]:
data.head()

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17893 entries, 0 to 17892
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17893 non-null  object
 1   mtype   17892 non-null  object
dtypes: object(2)
memory usage: 279.7+ KB


In [ ]:
inf = data.groupby('mtype').size().reset_index(name='Count').sort_values(by='Count', ascending=False)

In [ ]:
inf

,mtype,Count
11,Говядина,8422
33,Свинина,3050
23,Кура,1571
16,Индейка,1337
7,Баранина,1116
40,Цыпленок,942
22,Кролик,334
37,Утка,195
29,Оленина,193
20,Конина,176


In [ ]:
data = data.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

In [ ]:
data = data.query('mtype in ["баранина", "говядина", "индейка", "кура", "свинина", "цыпленок"]')

In [ ]:
data['mtype'].unique()

array(['баранина', 'индейка', 'говядина', 'свинина', 'кура', 'цыпленок'],
      dtype=object)

Выводы по анализу данных:
1. Есть позиции, где данные плохо заполнены, но их не так много + они не выведут какую-то категорию в значение 500 наименований
2. после обработки данных у нас получается следующее количество категорий: 
    * баранина
    * говядина
    * индейка
    * кура
    * свинина
    * цыпленок

In [ ]:
# Выбираем модель классификации
class ProductClassificationModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ProductClassificationModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded)
        output = self.fc(output[:, -1, :])
        return output


In [ ]:
# задаем классы датасета
class ProductDataset(Dataset):
    def __init__(self, data, labels, char_to_index):
        self.data = data
        self.labels = labels
        self.char_to_index = char_to_index
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text = self.data[index]
        label = self.labels[index]
        
        # Convert text to numerical sequence
        sequence = [self.char_to_index[char] for char in text]
        
        return torch.tensor(sequence, dtype=torch.long), torch.tensor(label, dtype=torch.long)


In [ ]:
# определеяем данные и необходимые классы
train_data = data['text'].tolist()
train_labels = data['mtype'].tolist()

In [ ]:
# создаем словарь из букв и присваиваем индексы
chars = sorted(list(set("".join(train_data))))
char_to_index = {char: index for index, char in enumerate(chars)}

In [ ]:
# конвертируем в векторные значения
selected_products = ["баранина", "говядина", "индейка", "кура", "свинина", "цыпленок"]
label_to_index = {label: index for index, label in enumerate(selected_products)}
train_labels = [label_to_index[label] for label in train_labels]

In [ ]:
# разбиваем на трейн и тест
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)


In [ ]:
# задаем параметры модели
input_size = len(chars)
hidden_size = 128
num_classes = len(selected_products)

In [ ]:
# определяем модель
model = ProductClassificationModel(input_size, hidden_size, num_classes)

In [ ]:
# задаем функцию потерь и выбираем оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

In [ ]:
# Create the dataset and dataloader
#train_dataset = ProductDataset(train_data, train_labels, char_to_index)
#train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: (pad_sequence([i[0] for i in x], batch_first=True), torch.stack([i[1] for i in x])))


In [ ]:
# преобразуем наши трейн и тест датасеты
train_dataset = ProductDataset(train_data, train_labels, char_to_index)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                              collate_fn=lambda x: (pad_sequence([i[0] for i in x], batch_first=True), torch.stack([i[1] for i in x])))

test_dataset = ProductDataset(test_data, test_labels, char_to_index)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, 
                             collate_fn=lambda x: (pad_sequence([i[0] for i in x], batch_first=True), torch.stack([i[1] for i in x])))


In [ ]:
%%time
# обучаем модель
num_epochs = 2 
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


Epoch 1/2, Loss: 407.30579176545143
Epoch 2/2, Loss: 324.6654616892338
CPU times: user 11min 49s, sys: 9.87 s, total: 11min 58s
Wall time: 12min 10s


In [ ]:
unknown_char_index = len(char_to_index) + 1

In [ ]:
# тестируем модель
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 83.19%


In [ ]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    
    text = re.sub(r"[^а-яА-Яa-zA-Z\s]", "", text)

    text = text.lower()

    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def tokenize_and_encode(text):
    tokens = [char_to_index.get(char, unknown_char_index) for char in text]
    return tokens





In [ ]:
# тестируем доп выборку
test_samples = [
    "Свинина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную свинину 2 сорт (80/20). Свободный объем 8 тонн. Самовывоз или доставка. Все подробности по телефону.",
    "Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:",
    "Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем."
]

# создаем для неё датафрейм
df = pd.DataFrame({"text": test_samples})

# Для увеличения точности работы модели проводим обработку предварительную
df["text"] = df["text"].apply(preprocess_text)
df["encoded_text"] = df["text"].apply(tokenize_and_encode)
class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe["encoded_text"].tolist()

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

# преобразовываем датасет
test_dataset = TextDataset(df)

# задаем параметры
batch_size = 1
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# переводим модель в режим предсказания, а не обучения
model.eval()

# создаем куда предсказывать
predictions = []

model.eval()
with torch.no_grad():
    for inputs in test_dataloader:
        inputs = torch.tensor(inputs, dtype=torch.long)  
        inputs = inputs.unsqueeze(0)  
        outputs = model(inputs)
        predicted_labels = [selected_products[index.item()] for index in outputs.argmax(dim=1)]
        predictions.extend(predicted_labels)

df["predicted_product"] = predictions

print(df)


                                                text  \
0  свинина блочная сорт в наличии ооо агросоюз ре...   
1  куриная разделка продам кур и куриную разделку...   
2  говяжью мукозу продам говяжью мукозу в охл и з...   

                                        encoded_text predicted_product  
0  [97, 82, 88, 93, 88, 93, 80, 2, 81, 91, 94, 10...           свинина  
1  [90, 99, 96, 88, 93, 80, 111, 2, 96, 80, 87, 8...              кура  
2  [83, 94, 82, 111, 86, 108, 110, 2, 92, 99, 90,...          говядина  


Выводы:
1. Аккураси на тестовой выборке 80%+, значение можно увеличить, но тогда модель будет значительно дольще обучаться на текущих мощностях, сейчас lr = 0.05, при значении 0.1 был 50%, вариант улучшения модели по гиперпараметрам - lr = 0.01 и ниже, так же увеличение количество эпох может позволить улучшить результат
2. при попадании новых данных для разделения на категории можно ещё сильнее улучшить их предварительную обработку функциями, наприме, была идея на тестовых фразах добавить лемматизацию, необходимо обсудить этот момент с тим лидом/старшим дс специалистом